In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
def loadpm():
    idx=1
    idx=str(idx)
    imgpath='pics/type/pichu/poke'+idx+'.jpg'
    try:
        pmimg=plt.imread(imgpath)
        pmimg=pmimg.reshape(1,pmimg.shape[0], pmimg.shape[1], 3)
    except:
        return
    idx=int(idx)
    idx+=1
    while True:
        idx=str(idx)
        imgpath='pics/type/pichu/poke'+idx+'.jpg'
        #imgpath='pics/type/grass/40x40/poke'+idx+'.jpg'
        try:
            img=plt.imread(imgpath)
        except:
            break
        pmimg =np.vstack((pmimg,img.reshape(1,img.shape[0], img.shape[1], 3)))
        idx=int(idx)
        idx+=1
    return pmimg

In [2]:
pokemon=loadpm()
pokemon=(pokemon-127.5)/127.5
print((pokemon.shape))

(151, 40, 40, 3)


In [3]:
import os
import numpy as np
import tensorflow as tf
from IPython.core.debugger import Tracer
from keras.layers import Input, Dense, Reshape, Flatten, Dropout,Conv2D,MaxPooling2D,Conv2DTranspose
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam,RMSprop,SGD

import matplotlib.pyplot as plt


Using TensorFlow backend.


In [4]:
class Generator():
    def __init__(self,gen_dim,output_size):
        self.network=Sequential()
        element=1
        for i in output_size:
            element*=i
        
        self.network.add(Dense(element//1*128,input_dim=gen_dim,activation='relu'))
        self.network.add(Reshape((output_size[0]//1,output_size[1]//1,output_size[2]*128)))
        '''self.network.add(BatchNormalization(momentum=0.8))
        #self.network.add(LeakyReLU(0.2))
        
        self.network.add(Dense(element//2,activation='relu'))
        self.network.add(BatchNormalization(momentum=0.8))
        self.network.add(Dense(element,activation='tanh'))
        
        self.network.add(Reshape(output_size))'''
        
        self.network.add(Conv2DTranspose(filters=256,kernel_size=(5,5),strides=1,padding='same'))
        self.network.add(BatchNormalization(momentum=0.8))
        self.network.add(LeakyReLU(0.2))
        self.network.add(Conv2DTranspose(filters=256,kernel_size=(3,3),strides=1,padding='same'))
        self.network.add(BatchNormalization(momentum=0.8))
        self.network.add(LeakyReLU(0.2))
        self.network.add(Conv2DTranspose(filters=3,kernel_size=(3,3),padding='same',activation='tanh'))
        
        self.network.summary()
class Discriminator():
    def __init__(self,input_size):
        self.network=Sequential()
        element=1
        for i in input_size:
            element*=i
        self.network.add(Conv2D(filters=256,kernel_size=(5,5),padding='same',input_shape=input_size))
        self.network.add(BatchNormalization(momentum=0.8))
        #self.network.add(LeakyReLU(0.2))
        self.network.add(Conv2D(filters=256,kernel_size=(3,3),padding='same'))
        self.network.add(BatchNormalization(momentum=0.8))
        #self.network.add(LeakyReLU(0.2))
        self.network.add(Conv2D(filters=256,kernel_size=(3,3),padding='same'))
        self.network.add(BatchNormalization(momentum=0.8))
        #self.network.add(LeakyReLU(0.2))
        self.network.add((Flatten()))
        #self.network.add(Dropout(0.25))
        self.network.add(Dense(20))
       
        self.network.add(Dense(1))
        optimizer=Adam(lr=0.0002, beta_1=0.5, decay=8e-8)
        self.network.compile(optimizer='SGD', loss='binary_crossentropy')
        self.network.summary()
class GAN():
    def __init__(self,trainingset,gen_dim):
        self.trainshape=trainingset.shape
        self.trainingset=trainingset
        self.gen_dim=gen_dim
        self.generator=Generator(gen_dim,(trainingset.shape[1],trainingset.shape[2],trainingset.shape[3]))
        self.discriminator=Discriminator((trainingset.shape[1],trainingset.shape[2],trainingset.shape[3]))
        self.discriminator.network.trainable = False
        self.GAN_input=Input((gen_dim,))
        self.GAN_output=self.discriminator.network(self.generator.network(self.GAN_input))
        self.network=Model(self.GAN_input,self.GAN_output)
        optimizer=Adam(lr=0.0002, beta_1=0.5, decay=8e-8)
        self.network.compile(optimizer=optimizer, loss='binary_crossentropy')
    def train(self,iterations):
        #train discriminator
        self.start=0
        self.batch_size=20
        self.more_to_stop=0
        for step in range(iterations):
        #prepare img
            self.random_gen_vectors=np.random.normal(size=(self.batch_size,self.gen_dim))
            self.generated_img=self.generator.network.predict(self.random_gen_vectors)
            self.stop=self.start+self.batch_size
            if self.stop>self.trainshape[0]:
                self.more_to_stop=int(self.stop-self.trainshape[0])
                self.stop=self.trainshape[0]
            self.real_img=self.trainingset[self.start:self.stop]
            if self.more_to_stop!=0:
                self.real_img=np.concatenate([self.real_img,self.trainingset[0:self.more_to_stop]])
                self.start=self.more_to_stop
                self.more_to_stop=0
            self.combined_img = np.concatenate([self.generated_img,self.real_img])
        #label img
            self.labels=np.concatenate([np.ones((self.batch_size, 1)),np.zeros((self.batch_size, 1))])
            #self.labels -= 0.05 * np.random.rand(self.labels.shape[0],self.labels.shape[1])
            self.labels=abs(self.labels)
            self.d_loss = self.discriminator.network.train_on_batch(self.combined_img,self.labels)
            self.start+=self.batch_size
            if(self.start>self.trainshape[0]-1):
                self.start=0
        #train generator
            self.mislabeled=np.zeros((self.batch_size,1))
            self.random_gen_vector=np.random.normal(size=(self.batch_size,self.gen_dim))
            self.g_loss=self.network.train_on_batch(self.random_gen_vector,self.mislabeled)
            print('iter:%d,d_loss=%f,g_loss=%f'%(step,self.d_loss,self.g_loss))
            if step%50==0:
                #save image
                random_vector=np.random.normal(size=(25,self.gen_dim))
                images=self.generator.network.predict(random_vector)
                images=(images)*127.5+127.5
                plt.figure(figsize=(10, 10))
                for i in range(images.shape[0]):
                    plt.subplot(5,5,i+1)
                    image = images[i, :, :, :]
                    image = np.reshape(image,(self.trainshape[1:]))
                    plt.imshow(image.astype(np.uint8))
                    plt.axis('off')
                plt.tight_layout()
                plt.savefig("output/poke_%d.png" % step)
                plt.close('all')
                    
gan=GAN(pokemon,20)
gan.train(20000)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 614400)            12902400  
_________________________________________________________________
reshape_1 (Reshape)          (None, 40, 40, 384)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 40, 40, 256)       2457856   
_________________________________________________________________
batch_normalization_1 (Batch (None, 40, 40, 256)       1024      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 40, 40, 256)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 40, 40, 256)       590080    
_________________________________________________________________
batc

C:\Users\a1091\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


iter:0,d_loss=3.744668,g_loss=8.768312
iter:1,d_loss=6.815049,g_loss=5.579835
iter:2,d_loss=7.218001,g_loss=7.971193
iter:3,d_loss=6.017929,g_loss=5.579835
iter:4,d_loss=5.606191,g_loss=7.174073
iter:5,d_loss=3.201655,g_loss=7.971193
iter:6,d_loss=3.201655,g_loss=7.174073
iter:7,d_loss=7.612167,g_loss=7.971193
iter:8,d_loss=5.606191,g_loss=7.971193
iter:9,d_loss=5.212024,g_loss=6.376954
iter:10,d_loss=4.804679,g_loss=7.174074
iter:11,d_loss=7.620954,g_loss=7.174074
iter:12,d_loss=2.798703,g_loss=7.971193
iter:13,d_loss=4.406119,g_loss=7.971193
iter:14,d_loss=4.406119,g_loss=6.376954
iter:15,d_loss=6.416489,g_loss=7.174073
iter:16,d_loss=4.813465,g_loss=8.768312


KeyboardInterrupt: 

In [ ]:
#gan.network.save_weights('_weights.h5')


In [ ]:
from keras.models import load_model
#gan.network.save('grass_model.h5') 
#gan.network=load_model('weights/water_model.h5')

In [ ]:
gan.network.summary()
gan.generator.network=gan.network.layers[1]
gan.discriminator.network=gan.network.layers[2]

In [ ]:
random_vector=np.random.normal(size=(25,25))
images=gan.generator.network.predict(random_vector)
images=(images+1)*127.5
plt.figure(figsize=(10, 10))
for i in range(images.shape[0]):
    plt.subplot(5,5,i+1)
    image = images[i, :, :, :]
    image = np.reshape(image,((40,40,3)))
    plt.imshow(image.astype(np.uint8),cmap='gray')
    plt.axis('off')
plt.tight_layout()
#plt.savefig("output/poke_%d.png" % step)
#plt.close('all')